In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical
import PIL 
import os
import cv2
from PIL import Image 
from numpy import asarray 

In [63]:
directory  = "./DataSet"

In [64]:
len_of_word = 5

In [65]:
vec =[]
c = '0'
for i in range (10):
    vec.append(c)
    c = chr(ord(c) + 1) 
c = 'a'
for i in range (26):
    vec.append(c)    
    c = chr(ord(c) + 1)
print(vec)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [66]:
def class_indices(label):
    ans = np.zeros(len(label))
    j = 0
    for ind in label:
        for i in range (36):
            if(vec[i]==ind):
                ans[j] = i
                j+=1
                break
    #ans = np.asarray(ans).astype('float32')
    #print(ans)
    return ans

In [67]:
def parse_filepath(filepath):
    path, filename = os.path.split(filepath)
    filename, ext = os.path.splitext(filename)
    label = filename.split("_")
    p = class_indices(label)
    cnt=0
    return label

In [68]:
import glob


files = glob.glob(os.path.join(directory,"*.jpg"))
# print(files)
labels = list(files)
labels = list(map(parse_filepath,files))
# print(labels)
df = pd.DataFrame(labels)
df['filepath'] = files
df.columns = ['labels','filepath']
df.dropna()
df.head()
#df.describe()

,labels,filepath
0,2a6y6,./DataSet/2a6y6.jpg
1,2a2yx,./DataSet/2a2yx.jpg
2,2a5pf,./DataSet/2a5pf.jpg
3,2a5cn,./DataSet/2a5cn.jpg
4,2a6yy,./DataSet/2a6yy.jpg


In [69]:
# To get the size of one image file 

def img_size():
    x =files[0]
    image = Image.open(x)
    print(image.mode)
    return (image.size)   
        

In [70]:
def pass_Data(df,indices,training_bool,batch_size=16):
    images = []
    labels = []
    while True:
        for i in indices:
            r = df.iloc[i]
            file, label = r['filepath'], r['labels']
            #print(label)
            im = Image.open(file)
            im = im.resize((227, 227))
            im = np.array(im) / 255.0
            images.append(np.array(im))
            # 2a341 = (2,11,3,4,1)
            l = to_categorical(class_indices(label),36)
            labels = np.append(labels,l)
            # 9*36*5
            #(9*36*5,)
            #(9,36*5)
            if len(images) >= batch_size:
                row = int((int(labels.shape[0]))/(36*len_of_word))
                col = int((int(labels.shape[0]))/(row))          
                labels = labels.reshape(row,col)
                yield np.array(images), np.array(labels)
                images, labels = [], []
        if not training_bool:
            break

In [71]:
# Splitting the data set into test train and validation set

p = np.random.permutation(len(df)) # random images are distributed.
train_up_to = int(len(df) * 0.8) # the train and the test set is divided into 80:20 
train_idx = p[:train_up_to]
test_idx = p[train_up_to:]

# split train_idx further into training and validation set
train_up_to = int(train_up_to * 0.8) # the train set is further divided into the validation and the training data set
train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]

print('train count: %s, valid count: %s, test count: %s' % (
    len(train_idx), len(valid_idx), len(test_idx)))

train count: 5, valid count: 2, test count: 2


In [72]:
#  Generating the test and the validation data 

batch_size = 2# insert Number
valid_batch_size = 1 #inset Number 

# def pass_Data(df,indices,training_bool,batch_size=16)
train_gen = pass_Data(df,train_idx,True,batch_size = batch_size)
validation_gen = pass_Data(df,valid_idx,True,batch_size = valid_batch_size)


In [73]:
# To calculate the validation and the training steps

validation_steps = (len(valid_idx)//valid_batch_size)
train_steps = (len(train_idx)//batch_size)
print(validation_steps,train_steps)

2 2


In [74]:
#  Alex_net Model making
def Alexnet():
#     THe input shape is always 257*257*3 or 256*256*3 and the images should be reshaped into it, if it is not that currently
    image_shape = (227,227,3)
    np.random.seed(1000)
    #model = Sequential()
    AlexNet = Sequential()

#1st Convolutional Layer
    AlexNet.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

#4th Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

#5th Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
    AlexNet.add(Flatten())
# 1st Fully Connected Layer
    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
    AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
#Add Dropout
    AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
#Add Dropout
    AlexNet.add(Dropout(0.4))

#Output Layer
    AlexNet.add(Dense(36*5))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))
    

#Model Summary
    AlexNet.summary()
    AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
    #AlexNet.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
    history = AlexNet.fit(train_gen,steps_per_epoch= train_steps,epochs = 2,validation_data = validation_gen, validation_steps = validation_steps)
    return history

In [75]:
Alexnet()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 57, 57, 96)        34944     
_________________________________________________________________
batch_normalization_18 (Batc (None, 57, 57, 96)        384       
_________________________________________________________________
activation_18 (Activation)   (None, 57, 57, 96)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 29, 29, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 256)       614656    
_________________________________________________________________
batch_normalization_19 (Batc (None, 29, 29, 256)       1024      
_________________________________________________________________
activation_19 (Activation)   (None, 29, 29, 256)      

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[2,10] labels_size=[2,180]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-74-6583b0bcbce8>:70) ]] [Op:__inference_train_function_4171]

Function call stack:
train_function
